# Introduzione
In questo file troviamo un modello di rete neurale per la classificazione del dataset COVID-CXR4.\
IL dataset è stato scaricato da [Kaggle](https://www.kaggle.com/datasets/andyczhao/covidx-cxr2) e messo dentro la cartella datasets \(questa cartella è ignorata da git perchè il dataset è grosso)\
I modelli salvati si possono trovare sotto la cartella [models](models) in modo da poterli usare senza rifare l'addestramento.

Questo *interactive pyhon notebook* è suddiviso in 3 parti principali:
- **Dataset**: in cui viene caricato, modificato e salvato in una cache l'intero dataset di immagini.
- **Modello**: in cui vengono creati e addestrati l'autoencoder e il classificatore.
- **Contrastive Learning**: in cui viene applicata la tecnica di contrastive learning per migliorare gli embedding da passare al classificatore.

Ogni parte del notebook contiene anche dei grafici e immagini per mostrare come i vari modelli si comportano.

In questa prima parte vengono importati le varie librerie usate e vengono create le variabili globali.

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras import layers, models, optimizers
from keras.api.preprocessing.image import load_img, img_to_array

models_dir = '../models'
datasets_dir = '../datasets'

# Ensure the directories exist
os.makedirs(models_dir, exist_ok=True)
os.makedirs(datasets_dir, exist_ok=True)

# Dataset
Modifica e caricamento del dataset.
Il dataset usato in questo caso è il dataset [COVIDx CXR-4](https://www.kaggle.com/datasets/andyczhao/covidx-cxr2).

Le modifiche apportate sono:
- Rimozione dei canali di colore \(alcune immagini hanno per esempio delle scritte rosse\); quindi ogni immagine è in scala di grigio.
- Ridimensionamento a 224x224 \(molte immagini sono 1024x1024 ma ci sono anche di dimensioni diverse\)

Le immagini importate sono sottoforma di array di numpy a 8bit che poi vengono salvate in un file cache (~4GB).\
Il primo blocco di codice dichiara delle funzioni utili per la modifica del dataset. La funzione `covid_cxr_data` è quella responsabile per il caricamento dei dati.

In [ ]:
def load_cache(file) -> dict:
    try:
        return dict(np.load(file, allow_pickle=True).item())
    except Exception:
        return {}

def images_to_numpy(images: pd.Series, size: tuple[int, int]):
    np_images = np.zeros((len(images), *size, 1), dtype=np.uint8)
    for i, img_name in enumerate(tqdm(images)):
        img = load_img(img_name, target_size=size, color_mode='grayscale')
        np_images[i] = img_to_array(img, dtype=np.uint8).reshape((*size, 1))
    return np_images

def classes_to_numpy(classes: pd.Series):
    return np.array(pd.factorize(classes)[0])

def covid_cxr_data(size:tuple[int, int]=(256, 256)):
    directory = f"{datasets_dir}/covid_cxr"
    cache = f"{directory}/cache_{size[0]}x{size[1]}.npy"
    dataset = load_cache(cache)

    if len(dataset) == 0:
        types = ['train', 'val', 'test']
        all_files = []
        for t in types:
            df = pd.read_csv(f"{directory}/{t}.txt", delimiter=' ', header=None)
            df[1] = df[1].apply(lambda x: f"{directory}/{t}/{x}")
            all_files.append(df)

        df = pd.concat(all_files)
        df.columns = ['id', 'filename', 'class', 'source']
        images = images_to_numpy(df['filename'], size)
        predictions = classes_to_numpy(df['class'])

        train_tot = len(all_files[0])
        val_tot = train_tot + len(all_files[1])
        test_tot = val_tot + len(all_files[2])

        dataset['train'] = (images[:train_tot], predictions[:train_tot])
        dataset['val'] = (images[train_tot:val_tot], predictions[train_tot:val_tot])
        dataset['test'] = (images[val_tot:test_tot], predictions[val_tot:test_tot])

        np.save(cache, dataset)

    x_train, y_train = dataset['train'][0], dataset['train'][1]
    x_val, y_val = dataset['val'][0], dataset['val'][1]
    x_test, y_test = dataset['test'][0], dataset['test'][1]
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

def data_generator(x, y, batch_size=32):
    num_samples = x.shape[0]
    indices = np.arange(num_samples)
    while True:
        np.random.shuffle(indices)  # Mescola gli indici all'inizio di ogni epoca
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch = image_int_to_float(x[batch_indices])
            #batch_val = y[batch_indices].reshape(-1, 1)
            yield batch, batch

def image_int_to_float(image:np.ndarray):
    return image.astype(np.float32) / 255.0


Di seguito carichiamo il dataset usando le funzioni dichiarate precedentemente e mostriamo quanti dati sono stati caricati per ogni tipologia \(training, validation, test\).

In [ ]:
shape = (224, 224)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = covid_cxr_data(shape)
total_classes = len(np.unique(y_train))

print(f"Train: {x_train.shape}, {y_train.shape}")
print(f"Validation: {x_val.shape}, {y_val.shape}")
print(f"Test: {x_test.shape}, {y_test.shape}")

Di seguito viene mostrato quante classi ci sono e come sono distribuite all'interno del dataset.\
Come si può notare il training set è sbilanciato verso una classe e questo non aiuta per l'addestramento del classificatore.

In [ ]:
# Count the number of samples per class for each dataset
train_counts = np.bincount(y_train)
val_counts = np.bincount(y_val)
test_counts = np.bincount(y_test)

# Plot the counts
plt.figure(figsize=(10, 6))
x_labels = range(total_classes)
plt.bar(x_labels, train_counts, width=0.2, label='Train', align='center')
plt.bar([x + 0.25 for x in x_labels], val_counts, width=0.2, label='Validation', align='center')
plt.bar([x + 0.5 for x in x_labels], test_counts, width=0.2, label='Test', align='center')
plt.xticks(x_labels, [f"Class {i}" for i in x_labels])
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Number of Samples per Class in Each Dataset')
plt.legend()
plt.tight_layout()
plt.show()

# Modello
In questa sezione vediamo i modelli per l'autoencoding e per la classificazione.\
Per prima cosa definiamo le variabili in cui troviamo i modelli e gli eventuali parametri di essi.

In [ ]:
latent_space = 256
paths = {
    'encoder': f"{models_dir}/encoder_{shape[0]}x{shape[1]}.keras",
    'autoencoder': f"{models_dir}/autoencoder_{shape[0]}x{shape[1]}.keras",
    'classifier': f"{models_dir}/classifier_{shape[0]}x{shape[1]}.keras",
    'contrastive': f"{models_dir}/contrastive_{shape[0]}x{shape[1]}.keras",
    'history_autoencoder': f"{models_dir}/history_auto_{shape[0]}x{shape[1]}.npy",
    'history_classifier': f"{models_dir}/history_clf_{shape[0]}x{shape[1]}.npy",
    'history_contrastive': f"{models_dir}/history_con_{shape[0]}x{shape[1]}.npy"
}

### Autoencoder
Il primo modello creato è l'autoencoder e usa gli stessi principi delle CNN per creare una rappresentazione compatta delle immagini. Infatti il modello è composto da dei Convolutional Layer che, riducono la dimensione spaziale per aumentare la dimensionde dei filtri.\
L'encoder ha inoltre dei layer di BatchNormalization.

Questo modello è quello più lungo da addestrare solamente perchè ha abbastanza parametri e il dataset, essendo grande, non ci sta in memoria.\
Per queste ragioni la batch è abbastanza piccola.

In [ ]:
try:
    encoder = models.load_model(paths['encoder'])
    autoencoder = models.load_model(paths['autoencoder'])

    enc_space = encoder.output_shape[1]
    if enc_space != latent_space:
        print(f"Encoder latent space mismatch: {enc_space} != {latent_space}")
    latent_space = enc_space

except Exception as e:
    in_encoder = layers.Input(shape=(*shape, 1))
    x = layers.BatchNormalization()(in_encoder)
    x = layers.Conv2D(32, 3, padding='same', strides=2, activation='relu')(x)
    x = layers.Conv2D(64, 3, padding='same', strides=2, activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', strides=2, activation='relu')(x)
    x = layers.Conv2D(256, 3, padding='same', strides=2, activation='relu')(x)
    before_flatten = x.shape[1:]
    x = layers.Flatten()(x)
    x = layers.BatchNormalization()(x)
    flatten = x.shape[1]
    latent = layers.Dense(latent_space, activation='sigmoid')(x)
    encoder = models.Model(in_encoder, latent, name='encoder')

    in_decoder = layers.Input(shape=(latent_space,))
    x = layers.Dense(flatten, activation='relu')(in_decoder)
    x = layers.Reshape(before_flatten)(x)
    x = layers.Conv2DTranspose(256, 3, padding='same', strides=2, activation='relu')(x)
    x = layers.Conv2DTranspose(128, 3, padding='same', strides=2, activation='relu')(x)
    x = layers.Conv2DTranspose(64, 3, padding='same', strides=2, activation='relu')(x)
    x = layers.Conv2DTranspose(32, 3, padding='same', strides=2, activation='relu')(x)
    out_decoder = layers.Conv2DTranspose(1, 3, padding='same', activation='sigmoid')(x)
    decoder = models.Model(in_decoder, out_decoder, name='decoder')

    in_autoencoder = layers.Input(shape=(*shape, 1))
    encoder_out = encoder(in_autoencoder)
    decoder_out = decoder(encoder_out)
    autoencoder = models.Model(in_autoencoder, decoder_out, name='autoencoder')
    autoencoder.compile(optimizer=optimizers.Adam(), loss='mse')

    batch = 32
    epochs = 4
    batch_steps = len(x_train) // batch
    batch_val_steps = len(x_val) // batch
    gen_train = data_generator(x_train, y_train, batch)
    gen_val = data_generator(x_val, y_val, batch)

    history_auto = autoencoder.fit(gen_train, validation_data=gen_val,
                                   epochs=epochs, steps_per_epoch=batch_steps, validation_steps=batch_val_steps)

    autoencoder.save(paths['autoencoder'])
    encoder.save(paths['encoder'])
    np.save(paths['history_autoencoder'], history_auto.history)

### Classificatore
Il classificatore è un modello semplice con 2 layer densi e un layer finale per la classificazione con la softmax.\
Essendo i dati molto più piccoli le batch possono essere alte e si possono avere molte più epoche per far imparare.

Purtroppo essendo il dataset molto sbilanciato verso una classe l'addestramento viene influenzato negativamente se non si fanno delle correzioni.

In [ ]:
try:
    classifier = models.load_model(paths['classifier'])

except Exception:
    in_classifier = layers.Input(shape=(latent_space,))
    x = layers.Dense(64, activation='relu')(in_classifier)
    x = layers.Dense(64, activation='relu')(x)
    out_classifier = layers.Dense(total_classes, activation='softmax')(x)
    classifier = models.Model(in_classifier, out_classifier, name='classifier')
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    batch = 1024
    epochs = 100
    x_train_class = encoder.predict(x_train, verbose=0)
    x_val_class = encoder.predict(x_val, verbose=0)

    history_class = classifier.fit(x_train_class, y_train, validation_data=(x_val_class, y_val),
                                   epochs=epochs, batch_size=batch)

    classifier.save(paths['classifier'])
    np.save(paths['history_classifier'], history_class.history)

### Risultati
Di seguito i risultati dell'addestramento se è stato fatto, altrimenti vengono mostrati solo delle predizioni di alcuni dati di test.

In [ ]:
history = [
    np.load(paths['history_autoencoder'], allow_pickle=True).item(),
    np.load(paths['history_classifier'], allow_pickle=True).item()
]

plt.figure(figsize=(12, 5))
for i, h in enumerate(history):
    metric = list(h.keys())[0]
    plt.subplot(1, 2, i+1)
    plt.plot(h[metric], label=f'Training {metric}')
    plt.plot(h[f'val_{metric}'], label=f'Validation {metric}')
    plt.title(f'Model {metric}')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Choose N random images from the test set
total = 10
indices = np.random.choice(len(x_test), total, replace=False)
orig_img = image_int_to_float(x_test[indices])
orig_classes = y_test[indices]
pred_img = autoencoder.predict(orig_img, verbose=0)
pred_classes = classifier.predict(encoder.predict(orig_img, verbose=0), verbose=0)

# Plot the original and predicted images
plt.figure(figsize=(15, 3.5))
for i, _ in enumerate(indices):
    ax = plt.subplot(2, total, i + 1)
    plt.imshow(orig_img[i], cmap='gray')
    plt.title(f"Original: {orig_classes[i]}")
    plt.axis('off')

    pred = np.argmax(pred_classes[i])
    correct = pred == orig_classes[i]

    ax = plt.subplot(2, total, i + total + 1)
    plt.imshow(pred_img[i], cmap='gray')
    plt.title(f"Pred: {pred_classes[i][pred]:.2f}", color='green' if correct else 'red')
    plt.axis('off')
plt.show()

# Contrastive Learning
